_Lambda School Data Science — Big Data_

# AWS SageMaker

### Links

#### AWS
- The Open Guide to Amazon Web Services: EC2 Basics _(just this one short section!)_ https://github.com/open-guides/og-aws#ec2-basics
- AWS in Plain English https://www.expeditedssl.com/aws-in-plain-english
- Amazon SageMaker » Create an Amazon SageMaker Notebook Instance https://docs.aws.amazon.com/sagemaker/latest/dg/gs-setup-working-env.html
- Amazon SageMaker » Install External Libraries https://docs.aws.amazon.com/sagemaker/latest/dg/nbi-add-external.html 

`conda install -n python3 bokeh dask datashader fastparquet numba python-snappy`

#### Dask
- Why Dask? https://docs.dask.org/en/latest/why.html
- Use Cases https://docs.dask.org/en/latest/use-cases.html
- User Interfaces https://docs.dask.org/en/latest/user-interfaces.html

#### Numba
- A ~5 minute guide http://numba.pydata.org/numba-doc/latest/user/5minguide.html

## 1. Estimate pi
https://en.wikipedia.org/wiki/Approximations_of_π#Summing_a_circle's_area

### With plain Python

In [1]:
import random

def monte_carlo_pi(nsamples):
    acc = 0
    for _ in range(int(nsamples)):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

In [2]:
%%time
monte_carlo_pi(1e7)

CPU times: user 4.18 s, sys: 0 ns, total: 4.18 s
Wall time: 4.18 s


3.141562

### With Numba
http://numba.pydata.org/

In [3]:
from numba import njit

In [4]:
@njit
def monte_carlo_pi(nsamples):
    acc = 0
    for _ in range(int(nsamples)):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

In [6]:
%%time
monte_carlo_pi(1e7)

CPU times: user 133 ms, sys: 0 ns, total: 133 ms
Wall time: 133 ms


3.1411376

## 2. Loop a slow function

### With plain Python

In [7]:
from time import sleep

def slow_square(x):
    sleep(1)
    return x**2

In [8]:
%%time
[slow_square(n) for n in range(16)]

CPU times: user 1.52 ms, sys: 425 µs, total: 1.95 ms
Wall time: 16 s


[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225]

### With Dask
- https://examples.dask.org/delayed.html
- http://docs.dask.org/en/latest/setup/single-distributed.html

In [9]:
from dask import compute, delayed

In [12]:
%%time
compute(delayed(slow_square)(n) for n in range(16))

CPU times: user 6.87 ms, sys: 787 µs, total: 7.66 ms
Wall time: 1.01 s


([0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225],)

## 3. Analyze millions of Instacart orders

### Download data
https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2

In [15]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2019-02-25 18:20:24--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.136.246
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.136.246|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’

instacart_online_gr 100%[===================>] 196.03M  95.1MB/s    in 2.1s    

2019-02-25 18:20:26 (95.1 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’ saved [205548478/205548478]



In [16]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [17]:
%cd instacart_2017_05_01

/home/ec2-user/SageMaker/DS-Unit-3-Sprint-3-Big-Data/module1-aws-sagemaker/instacart_2017_05_01


In [18]:
!ls -lh *.csv

-rw-r--r-- 1 ec2-user ec2-user 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 ec2-user ec2-user  270 May  2  2017 departments.csv
-rw-r--r-- 1 ec2-user ec2-user 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 ec2-user ec2-user  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 ec2-user ec2-user 104M May  2  2017 orders.csv
-rw-r--r-- 1 ec2-user ec2-user 2.1M May  2  2017 products.csv


### With Pandas

#### Load & merge data

In [13]:
import pandas as pd

In [19]:
%%time
order_products = pd.concat([
    pd.read_csv('order_products__prior.csv'), 
    pd.read_csv('order_products__train.csv')])

order_products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33819106 entries, 0 to 1384616
Data columns (total 4 columns):
order_id             int64
product_id           int64
add_to_cart_order    int64
reordered            int64
dtypes: int64(4)
memory usage: 1.3 GB
CPU times: user 10.1 s, sys: 2.14 s, total: 12.2 s
Wall time: 12.2 s


In [20]:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [21]:
products = pd.read_csv('products.csv')
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 4 columns):
product_id       49688 non-null int64
product_name     49688 non-null object
aisle_id         49688 non-null int64
department_id    49688 non-null int64
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


In [22]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [23]:
%%time
order_products = pd.merge(order_products, products[['product_id', 'product_name']])

CPU times: user 7.74 s, sys: 2.17 s, total: 9.91 s
Wall time: 9.91 s


In [24]:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered,product_name
0,2,33120,1,1,Organic Egg Whites
1,26,33120,5,0,Organic Egg Whites
2,120,33120,13,0,Organic Egg Whites
3,327,33120,5,1,Organic Egg Whites
4,390,33120,28,1,Organic Egg Whites


#### Most popular products?

In [25]:
%%time
order_products['product_name'].value_counts()

CPU times: user 4.06 s, sys: 42.6 ms, total: 4.1 s
Wall time: 4.08 s


Banana                                          491291
Bag of Organic Bananas                          394930
Organic Strawberries                            275577
Organic Baby Spinach                            251705
Organic Hass Avocado                            220877
Organic Avocado                                 184224
Large Lemon                                     160792
Strawberries                                    149445
Limes                                           146660
Organic Whole Milk                              142813
Organic Raspberries                             142603
Organic Yellow Onion                            117716
Organic Garlic                                  113936
Organic Zucchini                                109412
Organic Blueberries                             105026
Cucumber Kirby                                   99728
Organic Fuji Apple                               92889
Organic Lemon                                    91251
Organic Gr

#### Organic?

In [26]:
%%time
order_products['organic'] = order_products['product_name'].str.contains('Organic')

CPU times: user 17.4 s, sys: 107 ms, total: 17.5 s
Wall time: 17.5 s


In [27]:
%%time
order_products['organic'].value_counts()

CPU times: user 277 ms, sys: 123 ms, total: 399 ms
Wall time: 398 ms


False    23163118
True     10655988
Name: organic, dtype: int64

### With Dask
https://examples.dask.org/dataframe.html

In [28]:
import dask.dataframe as dd
from dask.distributed import Client

client = Client(n_workers=16)
client

Client Scheduler: tcp://127.0.0.1:41599 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 16 Cores: 16 Memory: 67.53 GB


#### Load & merge data
https://examples.dask.org/dataframes/01-data-access.html#Read-CSV-files

In [29]:
%%time
order_products = dd.read_csv('order_products*.csv')

CPU times: user 21.7 ms, sys: 3.08 ms, total: 24.8 ms
Wall time: 22.2 ms


In [30]:
order_products

,order_id,product_id,add_to_cart_order,reordered
npartitions=11,,,,
,int64,int64,int64,int64
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


In [31]:
%%time
order_products = dd.merge(order_products, products[['product_id','product_name']])

CPU times: user 26.3 ms, sys: 4.24 ms, total: 30.6 ms
Wall time: 27.3 ms


http://docs.dask.org/en/latest/dataframe-performance.html#persist-intelligently

In [32]:
%%time
order_products = order_products.persist()

CPU times: user 6.26 ms, sys: 40 µs, total: 6.3 ms
Wall time: 6.01 ms


#### Most popular products?

In [34]:
%%time
order_products['product_name'].value_counts().compute()

CPU times: user 231 ms, sys: 37.6 ms, total: 269 ms
Wall time: 1.57 s


Banana                                                       491291
Bag of Organic Bananas                                       394930
Organic Strawberries                                         275577
Organic Baby Spinach                                         251705
Organic Hass Avocado                                         220877
Organic Avocado                                              184224
Large Lemon                                                  160792
Strawberries                                                 149445
Limes                                                        146660
Organic Whole Milk                                           142813
Organic Raspberries                                          142603
Organic Yellow Onion                                         117716
Organic Garlic                                               113936
Organic Zucchini                                             109412
Organic Blueberries                             

#### Organic?

In [36]:
%%time
order_products['organic'] = order_products['product_name'].str.contains('Organic')

CPU times: user 6.09 ms, sys: 124 µs, total: 6.21 ms
Wall time: 5.76 ms


In [38]:
%%time
order_products['organic'].value_counts().compute() / len(order_products)

CPU times: user 2.05 s, sys: 381 ms, total: 2.43 s
Wall time: 16 s


False    0.684912
True     0.315088
Name: organic, dtype: float64

## 4. Fit a machine learning model

### Load data

In [ ]:
%cd ../ds-predictive-modeling-challenge

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

train_features = pd.read_csv('train_features.csv')
train_labels = pd.read_csv('train_labels.csv')

X_train_numeric = train_features.select_dtypes(np.number)
y_train = train_labels['status_group']

### With 2 cores (like Google Colab)
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [ ]:
model = RandomForestClassifier(n_estimators=200, oob_score=True, n_jobs=2, random_state=42, verbose=1)
model.fit(X_train_numeric, y_train)
print('Out-of-bag score:', model.oob_score_)

### With 16 cores (on AWS m4.4xlarge)

## ASSIGNMENT

Revisit a previous assignment or project that had slow speeds or big data.

Make it better with what you've learned today!

You can use `wget` or Kaggle API to get data. Some possibilities include:

- https://www.kaggle.com/c/ds1-predictive-modeling-challenge
- https://www.kaggle.com/ntnu-testimon/paysim1
- https://github.com/mdeff/fma
- https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2 



Also, you can play with [Datashader](http://datashader.org/) and its [example datasets](https://github.com/pyviz/datashader/blob/master/examples/datasets.yml)!